## [LangChain Quickstart Guide](https://python.langchain.com/docs/get_started/quickstart)

Load the environmental variables form .env file

In [93]:
import json
import textwrap
from dotenv import load_dotenv
import os
load_dotenv()

True

In [64]:
def printlongtext(text, max_lines=10):
    short_strings = textwrap.wrap(text, 80)
    lines = len(short_strings)
    if lines < max_lines:
        max_lines = lines
    for index in range(0, max_lines):
        print(short_strings[index])
        

### LLM Chain

In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [7]:
result =llm.invoke("how can langsmith help with testing?")

Examine the result

In [23]:
print(result.content)
# print the json object as a string with pretty formatting with indent of 4, also make it wrap long lines
print(json.dumps(result.json(), indent=4))

Langsmith can help with testing in a few ways:

1. Automated testing: Langsmith can help create and execute automated tests, saving time and effort in manual testing.

2. Test case management: Langsmith can help organize and manage test cases, making it easier to track progress and ensure complete test coverage.

3. Test data generation: Langsmith can help generate test data to simulate real-world scenarios, improving the accuracy and effectiveness of testing.

4. Performance testing: Langsmith can help conduct performance testing to assess how well a system performs under various conditions.

5. Bug tracking: Langsmith can help track and manage bugs found during testing, making it easier to prioritize and resolve issues. 

Overall, Langsmith can streamline the testing process, improve efficiency, and enhance the overall quality of software products.
"{\"content\": \"Langsmith can help with testing in a few ways:\\n\\n1. Automated testing: Langsmith can help create and execute automate

In [4]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

Combine prompt and llm into a chain.

In [5]:
chain = prompt | llm 

In [6]:
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content='Langsmith can help with testing in several ways:\n\n1. Test Automation: Langsmith can automate the testing process by writing scripts and test cases in various programming languages. This can help in executing repetitive tests quickly and efficiently.\n\n2. Test Data Generation: Langsmith can generate test data for testing purposes. It can create a variety of data sets to simulate different scenarios and edge cases, helping to ensure thorough test coverage.\n\n3. Performance Testing: Langsmith can be used to conduct performance testing by simulating a large number of users or transactions. This can help in identifying any performance bottlenecks or issues in the system.\n\n4. Integration Testing: Langsmith can assist in integration testing by writing scripts to test the interactions between different components or systems. This can help in ensuring that the integrated system functions as expected.\n\n5. Regression Testing: Langsmith can automate regression testing by

In [24]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [25]:
chain = prompt | llm | output_parser

In [26]:
result = chain.invoke({"input": "how can langsmith help with testing?"})

'Langsmith can help with testing in several ways:\n\n1. Test Generation: Langsmith can automatically generate test cases based on the specified requirements and input parameters. This can save time and effort in manually creating test cases.\n\n2. Test Automation: Langsmith can automate the execution of test cases, making it easier to run tests repeatedly and consistently. This can help in regression testing and ensuring the stability of the software.\n\n3. Test Data Generation: Langsmith can generate test data to cover different scenarios and edge cases, ensuring thorough testing of the software.\n\n4. Code Analysis: Langsmith can perform code analysis to identify potential issues and bugs in the code, helping testers to focus on areas that require attention.\n\n5. Integration with Testing Tools: Langsmith can integrate with popular testing tools and frameworks, making it easier to incorporate automated testing into the development process.\n\nOverall, Langsmith can streamline the tes

In [31]:
print(json.dumps(result.json(), indent=4 ))
print("-" * 80)
print(result.content)

"{\"content\": \"Langsmith can help with testing in a few ways:\\n\\n1. Automated testing: Langsmith can help create and execute automated tests, saving time and effort in manual testing.\\n\\n2. Test case management: Langsmith can help organize and manage test cases, making it easier to track progress and ensure complete test coverage.\\n\\n3. Test data generation: Langsmith can help generate test data to simulate real-world scenarios, improving the accuracy and effectiveness of testing.\\n\\n4. Performance testing: Langsmith can help conduct performance testing to assess how well a system performs under various conditions.\\n\\n5. Bug tracking: Langsmith can help track and manage bugs found during testing, making it easier to prioritize and resolve issues. \\n\\nOverall, Langsmith can streamline the testing process, improve efficiency, and enhance the overall quality of software products.\", \"additional_kwargs\": {}, \"type\": \"ai\", \"example\": false}"
---------------------------

### Retrival Chain

WebBaseLoader uses beautifulsoup to scrape text from a web page.

In [32]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com")

docs = loader.load()

In [69]:
printlongtext(docs[0].page_content.strip("\n"), 20)
print("-" * 80)
print(json.dumps(docs[0].metadata, indent=4))

LangSmith | 🦜️🛠️ LangSmith      Skip to main content🦜️🛠️ LangSmith DocsLangChain
Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser
GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt
HubLangSmithOn this pageLangSmithIntroduction​LangSmith is a platform for
building production-grade LLM applications.It lets you debug, test, evaluate,
and monitor chains and intelligent agents built on any LLM framework and
seamlessly integrates with LangChain, the go-to open source framework for
building with LLMs.LangSmith is developed by LangChain, the company behind the
open source LangChain framework.Quick Start​Tracing: Get started with the
tracing quick start.Evaluation: Get started with the evaluation quick start.Next
Steps​Check out the following sections to learn more about LangSmith:User Guide:
Learn about the workflows LangSmith supports at each stage of the LLM
application lifecycle.Setup: Learn how to create an account, obtain an API key,


This time uses a local embedding model from ollama

In [34]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [35]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [46]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [47]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [73]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

LangSmith can help with testing by allowing users to debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework. It provides tracing and evaluation capabilities, as well as integration with the LangChain framework for building with LLMs. Users can also access the Prompt Hub, a prompt management tool built into LangSmith, for testing purposes.


In [76]:
printlongtext(response["answer"])

LangSmith can help with testing by allowing users to debug, test, evaluate, and
monitor chains and intelligent agents built on any LLM framework. It provides
tracing and evaluation capabilities, as well as integration with the LangChain
framework for building with LLMs. Users can also access the Prompt Hub, a prompt
management tool built into LangSmith, for testing purposes.


## Conversation Retrieval Chain

In [77]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [79]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
result = retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

In [85]:
printlongtext(result[0].page_content)

LangSmith | 🦜️🛠️ LangSmith      Skip to main content🦜️🛠️ LangSmith DocsLangChain
Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser
GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt
HubLangSmithOn this pageLangSmithIntroduction​LangSmith is a platform for
building production-grade LLM applications.It lets you debug, test, evaluate,
and monitor chains and intelligent agents built on any LLM framework and
seamlessly integrates with LangChain, the go-to open source framework for
building with LLMs.LangSmith is developed by LangChain, the company behind the
open source LangChain framework.Quick Start​Tracing: Get started with the
tracing quick start.Evaluation: Get started with the evaluation quick start.Next


In [86]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [87]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
result = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

In [91]:
printlongtext(result["answer"])

LangSmith can help test your LLM applications by providing tracing and
evaluation capabilities. With tracing, you can debug and monitor your chains and
intelligent agents built on any LLM framework. The evaluation feature allows you
to test and assess the performance of your applications. Additionally, LangSmith
seamlessly integrates with LangChain, the open source framework for building
with LLMs, to enhance the testing process.


## Agent

In [92]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [94]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [95]:
tools = [retriever_tool, search]

In [96]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [97]:
result = agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'How can LangSmith help with testing?'}`


[Document(page_content="LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\nSkip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction\u200bLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start\u200bTracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps\u200bCheck out the following sections to learn more abou

In [105]:
# print the json object as a string with pretty formatting with indent of 4, also make it wrap long lines
print(json.dumps(result, indent=4))
print("-" * 80)
printlongtext(result["output"], 20)

{
    "input": "how can langsmith help with testing?",
    "output": "LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It can help with testing by allowing you to debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework. LangSmith seamlessly integrates with LangChain, which is an open-source framework for building with LLMs. \n\nYou can use LangSmith for tracing, evaluation, and monitoring your LLM applications. It supports various workflows at each stage of the LLM application lifecycle. Additionally, LangSmith offers a Prompt Hub, which is a prompt management tool built into the platform.\n\nIf you are interested in learning more about how LangSmith can specifically help with testing, you can explore the User Guide, Setup, and Evaluation sections on the LangSmith platform."
}
--------------------------------------------------------------------------------
LangSmith is a platform for building production-g

In [106]:
sf_weather = agent_executor.invoke({"input": "what is the weather in SF?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://www.wunderground.com/forecast/us/ca/san-francisco', 'content': 'Get the latest weather information for San Francisco, CA, including temperature, precipitation, wind speed, and humidity. See the hourly and 10-day forecast for the South of Market station and other nearby weather stations.'}, {'url': 'https://www.accuweather.com/en/us/san-francisco/94103/weather-forecast/347629', 'content': 'Get the current weather, radar, and air quality for San Francisco, CA. See the forecast for today, tonight, and the next two days, as well as the outlook for the next week.'}, {'url': 'https://www.theweathernetwork.com/us/weather/california/san-francisco', 'content': 'Get the most current and reliable weather information for San Francisco, California, including storm alerts, reports and videos. See the temperature, feels like, wind, humidity, visibility and m

In [107]:
printlongtext(sf_weather["output"], 20)

You can check the latest weather information for San Francisco, CA, including
temperature, precipitation, wind speed, and humidity on websites like [Weather
Underground](https://www.wunderground.com/forecast/us/ca/san-francisco),
[AccuWeather](https://www.accuweather.com/en/us/san-francisco/94103/weather-
forecast/347629), [The Weather
Network](https://www.theweathernetwork.com/us/weather/california/san-francisco),
and [Weather.com](https://weather.com/weather/hourbyhour/l/USCA0987:1:US).


In [108]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
result_ls = agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith LLM application testing services'}`


[Document(page_content="LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\nSkip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction\u200bLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start\u200bTracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps\u200bCheck out the following sections to learn mor

In [109]:
printlongtext(result_ls["output"], 20)

LangSmith is a platform for building production-grade LLM applications. It
allows you to debug, test, evaluate, and monitor chains and intelligent agents
built on any LLM framework. LangSmith seamlessly integrates with LangChain, the
open-source framework for building with LLMs.   You can learn more about
LangSmith by checking out their user guide, setup instructions, pricing details,
self-hosting options, tracing capabilities, evaluation capabilities, and the
Prompt Hub. Additionally, you can explore the LangSmith Cookbook, which is a
collection of tutorials and end-to-end walkthroughs using LangSmith.  If you are
interested in using LangSmith for testing your LLM applications, you can follow
the quick start guides for tracing and evaluation provided by LangSmith.


## Serving with LangServe

Refer to serv.py